matrix transformation only for perfect file (with path 0)

In [9]:
import pandas as pd
import os

from positionism.read import parameter
from positionism.optimizer import opti_amount_weirdos
from positionism.plot import optimizer

direc = os.getcwd() # get current working directory

Input

In [10]:
file_perfect_poscar_48n24_wo_cif = "Li6PS5Cl_type0_init"
litype = 0
litype_site = "24g" # "24g", "48h1", "48h2"

activate_radius = 1
max_mapping_radius = 0.043
max_mapping_radius_48htype2 = 0.066
max_mapping_radius_48htype1_48htype2 = (max_mapping_radius + max_mapping_radius_48htype2) / 2

delta_coor = 0.002
range_coor = 0.005

In [11]:
# Replace with your file path
file_perfect_poscar_48n24 = f"{file_perfect_poscar_48n24_wo_cif}.cif"

reference_folder = "_reference_cif"
results_folder = "_results"
iter_type = "varying_position"
file_path_ori_ref_48n24 = f"./{reference_folder}/{file_perfect_poscar_48n24}"
folder_results_iter_type = f"/{results_folder}/_{iter_type}/{file_perfect_poscar_48n24_wo_cif}/"

n_file_start = 0
n_file_end = 54

In [12]:
if activate_radius == 1:
    df_mapping_metainfo = pd.read_pickle(f'df_mapping_metainfo_{max_mapping_radius}_{file_perfect_poscar_48n24_wo_cif}.pkl') 
elif activate_radius == 2:
    df_mapping_metainfo = pd.read_pickle(f'df_mapping_metainfo_{max_mapping_radius}_{max_mapping_radius_48htype2}_{file_perfect_poscar_48n24_wo_cif}.pkl')
elif activate_radius == 3:
    df_mapping_metainfo = pd.read_pickle(f'df_mapping_metainfo_{max_mapping_radius}_{max_mapping_radius_48htype2}_{max_mapping_radius_48htype1_48htype2}_{file_perfect_poscar_48n24_wo_cif}.pkl')

df_wo_weirdos_selectedcol = df_mapping_metainfo[['geometry','path','subdir_new_system', 'toten [eV]']].sort_values("toten [eV]", ascending=True)[n_file_start:n_file_end].reset_index().copy()

In [13]:
max_mapping_radius_init = max_mapping_radius
max_mapping_radius_48htype2_init = max_mapping_radius_48htype2

ref_positions_array = parameter.x_y_z(file_path_ori_ref_48n24, litype)

if litype == 0:
    x, y, z = ref_positions_array[0:3]
elif litype == 1:
    x, y, z = ref_positions_array[3:6]
elif litype == 2:
    x, y, z = ref_positions_array[6:9]
elif litype == 3:
    x, y, z = ref_positions_array[9:12]
elif litype == 4:
    x, y, z = ref_positions_array[12:15]
elif litype == 5:
    x, y, z = ref_positions_array[15:18]
elif litype == 6:
    x, y, z = ref_positions_array[18:21]
elif litype == 7:
    x, y, z = ref_positions_array[21:24]
elif litype == 8:
    x, y, z = ref_positions_array[24:27]
else:
    print("litype is out of range")


# idea: outer loop dz2 positive, then dz1 ositive and negative
# 2nd loop: dz2 negative

In [14]:
max_coor_x = x + range_coor + delta_coor
# min_coor_x = x - range_coor - delta_coor
min_coor_x = -a * max_coor_x

min_coor_y = y - range_coor - delta_coor
max_coor_y = y + range_coor + delta_coor

min_coor_z = z - range_coor - delta_coor
max_coor_z = z + range_coor + delta_coor

In [15]:
# Define constants
col_x = "x"
col_y = "y"
col_z = "z"
col_sumweirdosLi = "sumweirdosLi"

# Initialize DataFrame
df_x_y_z_sumweirdosLi = pd.DataFrame(columns=[col_x, col_y, col_z, col_sumweirdosLi])

# Global index
idx_df = 0

# Set to track seen (x, y, z) combinations
seen_combinations = set()

# Function to update DataFrame
def update_dataframe(next_x, next_y, next_z, sum_weirdos_Li):
    global idx_df
    df_x_y_z_sumweirdosLi.at[idx_df, col_x] = next_x
    df_x_y_z_sumweirdosLi.at[idx_df, col_y] = next_y
    df_x_y_z_sumweirdosLi.at[idx_df, col_z] = next_z
    df_x_y_z_sumweirdosLi.at[idx_df, col_sumweirdosLi] = sum_weirdos_Li
    idx_df += 1

# Function to iterate over z and x
def iterate_z_x(start_x, step_x, next_y, start_z, end_z, step_z):
    next_x = start_x
    next_z = start_z
    while (step_z > 0 and next_z <= end_z) or (step_z < 0 and next_z >= end_z):
        ref_positions_array = [next_x, next_y, next_z]
        
        # Check if combination has already been processed
        if (next_x, next_y, next_z) not in seen_combinations:
            sum_weirdos_Li = opti_amount_weirdos.get_sum_weirdos_Li(
                ref_positions_array, max_mapping_radius, max_mapping_radius_48htype2, 
                df_wo_weirdos_selectedcol, activate_radius, file_perfect_poscar_48n24_wo_cif, 
                litype, iter_type
            )
            print(f"x: {next_x}, y: {next_y}, z: {next_z}, idx: {idx_df}")
            update_dataframe(next_x, next_y, next_z, sum_weirdos_Li)
            seen_combinations.add((next_x, next_y, next_z))

        next_z += step_z
        next_x += step_x


# Function to iterate over y and z
def iterate_y_z(next_x, start_y, step_y, start_z, end_z, step_z):
    next_y = start_y
    next_z = start_z
    while (step_z > 0 and next_z <= end_z) or (step_z < 0 and next_z >= end_z):
        ref_positions_array = [next_x, next_y, next_z]
        
        # Check if combination has already been processed
        if (next_x, next_y, next_z) not in seen_combinations:
            sum_weirdos_Li = opti_amount_weirdos.get_sum_weirdos_Li(
                ref_positions_array, max_mapping_radius, max_mapping_radius_48htype2, 
                df_wo_weirdos_selectedcol, activate_radius, file_perfect_poscar_48n24_wo_cif, 
                litype, iter_type
            )
            print(f"x: {next_x}, y: {next_y}, z: {next_z}, idx: {idx_df}")
            update_dataframe(next_x, next_y, next_z, sum_weirdos_Li)
            seen_combinations.add((next_x, next_y, next_z))

        next_z += step_z
        next_y += step_y


if litype_site == "24g":
    # Main loop function
    def main_loop():
        global next_x
        # Loop 1: All forward
        next_x = x
        while next_x <= max_coor_x:
            iterate_y_z(next_x, y, delta_coor, z, max_coor_z, delta_coor)
            next_x += delta_coor

        # Loop 2: Inner loop backward
        next_x = x
        while next_x <= max_coor_x:
            iterate_y_z(next_x, y, -delta_coor, z, min_coor_z, -delta_coor)
            next_x += delta_coor

        # Loop 3: All backward
        next_x = x
        while next_x >= min_coor_x:
            iterate_y_z(next_x, y, -delta_coor, z, min_coor_z, -delta_coor)
            next_x -= delta_coor

        # Loop 4: Outer backward
        next_x = x
        while next_x >= min_coor_x:
            iterate_y_z(next_x, y, delta_coor, z, max_coor_z, delta_coor)
            next_x -= delta_coor

elif litype_site == "48h1":
    # Main loop function
    def main_loop():
        global next_y
        # Loop 1: All forward
        next_y = y
        while next_y <= max_coor_y:
            iterate_z_x(x, -delta_coor, next_y, z, max_coor_z, delta_coor)
            next_y += delta_coor

        # Loop 2: Inner loop backward
        next_y = y
        while next_y <= max_coor_y:
            iterate_z_x(x, delta_coor, next_y, z, min_coor_z, -delta_coor)
            next_y += delta_coor

        # Loop 3: All backward
        next_y = y
        while next_y >= min_coor_y:
            iterate_z_x(x, delta_coor, next_y, z, min_coor_z, -delta_coor)
            next_y -= delta_coor

        # Loop 4: Outer backward
        next_y = y
        while next_y >= min_coor_y:
            iterate_z_x(x, -delta_coor, next_y, z, max_coor_z, delta_coor)
            next_y -= delta_coor

elif litype_site == "48h2":
    # Main loop function
    def main_loop():
        global next_y
        # Loop 1: All forward
        next_y = y
        while next_y <= max_coor_y:
            iterate_z_x(x, delta_coor, next_y, z, max_coor_z, delta_coor)
            next_y += delta_coor

        # Loop 2: Inner loop backward
        next_y = y
        while next_y <= max_coor_y:
            iterate_z_x(x, -delta_coor, next_y, z, min_coor_z, -delta_coor)
            next_y += delta_coor

        # Loop 3: All backward
        next_y = y
        while next_y >= min_coor_y:
            iterate_z_x(x, -delta_coor, next_y, z, min_coor_z, -delta_coor)
            next_y -= delta_coor

        # Loop 4: Outer backward
        next_y = y
        while next_y >= min_coor_y:
            iterate_z_x(x, delta_coor, next_y, z, max_coor_z, delta_coor)
            next_y -= delta_coor


In [16]:
# Run the main loop
main_loop()

# Display the resulting DataFrame
print(df_x_y_z_sumweirdosLi)

/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

x: 0.02311, y: 0.25, z: 0.25, idx: 0


/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

x: 0.02311, y: 0.27, z: 0.27, idx: 1


/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

...

x: 0.02311, y: 0.31000000000000005, z: 0.31000000000000005, idx: 3


/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

x: 0.043109999999999996, y: 0.25, z: 0.25, idx: 4


/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_subdir_positive_file][idx] = destination_path
/home/azka/Documents/01_Masterthesis/Atomic-Positionism/positionism/preprocessing/CONTCARorPOSCAR.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

KeyboardInterrupt: 

In [ ]:
min_sumweirdos = int(df_x_y_z_sumweirdosLi["sumweirdosLi"].min())

In [ ]:
if activate_radius == 2:
    df_x_y_z_sumweirdosLi.to_excel(f'08_varying_dx_dz_{min_sumweirdos}_{max_mapping_radius}_{max_mapping_radius_48htype2}_{file_perfect_poscar_48n24_wo_cif}.xlsx', index=False)
elif activate_radius == 1:
    df_x_y_z_sumweirdosLi.to_excel(f'08_varying_dx_dz_{min_sumweirdos}_{max_mapping_radius}_{file_perfect_poscar_48n24_wo_cif}.xlsx', index=False)

In [ ]:
df_x_y_z_sumweirdosLi = pd.read_excel("08_varying_dx_dz_435_0.043_Li6PS5Cl_type0_init.xlsx")

In [ ]:
%matplotlib inline

optimizer.heatmap_x_y(df_x_y_z_sumweirdosLi, folder_results_iter_type)